In [1]:
#tweet cleaning
#https://www.kaggle.com/ragnisah/text-data-cleaning-tweets-analysis
#https://towardsdatascience.com/basic-data-cleaning-engineering-session-twitter-sentiment-data-95e5bd2869ec

#Topic Modeilling
#https://towardsdatascience.com/end-to-end-topic-modeling-in-python-latent-dirichlet-allocation-lda-35ce4ed6b3e0

#Cleaning
#Puncuation
#lower case
#stopwords
#abbreviations

# Importing modules
import pandas as pd
import os

# Load the regular expression library
import re

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import nltk as nltk
nltk.download('stopwords')
nltk.download('wordnet')
import gensim
from gensim.utils import simple_preprocess
from gensim.models.wrappers import LdaMallet
from nltk.corpus import stopwords
import string
import re
%matplotlib inline
pd.set_option('display.max_colwidth', 100)
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

import pyLDAvis.gensim
import pickle 
import pyLDAvis
#import spacy
#from scispacy.abbreviation import AbbreviationDetector

#sentiment analysis
#https://medium.com/swlh/simple-sentiment-analysis-for-nlp-beginners-and-everyone-else-using-vader-and-textblob-728da3dbe33d
import sqlite3
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\megha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\megha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
C:\Users\megha\anaconda3\lib\site-packages\sklearn\decomposition\_lda.py:28: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  EPS = np.finfo(np.float).eps


In [2]:
warnings.filterwarnings("ignore",category=DeprecationWarning)

C:\Users\megha\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
diabetes2018 = pd.read_csv('diabetes2018_lda.csv')

In [4]:
# Print head
diabetes2018.head(10)

,Post ID,Publish Date,Author,Text,text_gens_nonstop,text_processed,text_tokenized,text_nlk_nonstop,text_nonstop_spacy,text_stemmed,text_lemmatized,Dominant,month_year
0,7ng7rl,2018-01-01 09:16:00,penguinsarefun,Hey all! Happy New year's! Just wanted to say hi. I was diagnosed with type 2 DM on Thursday. ...,Hey all! Happy New year's! Just wanted hi. I diagnosed type 2 DM Thursday. I've crazy fatigued t...,hey all happy new years just wanted hi i diagnosed type dm thursday ive crazy fatigued thirsty ...,"['hey', 'all', 'happy', 'new', 'years', 'just', 'wanted', 'hi', 'i', 'diagnosed', 'type', 'dm', ...","['happy', 'new', 'years', 'wanted', 'diagnosed', 'type', 'dm', 'thursday', 'crazy', 'fatigued', ...","['happy', 'new', 'years', 'wanted', 'diagnosed', 'type', 'dm', 'thursday', 'crazy', 'fatigued', ...","['happi', 'new', 'year', 'want', 'diagnos', 'type', 'dm', 'thursday', 'crazi', 'fatigu', 'thirst...","['happy', 'new', 'year', 'wanted', 'diagnosed', 'type', 'dm', 'thursday', 'crazy', 'fatigued', '...",Topic 2,2018-01
1,7nlbda,2018-01-02 00:58:00,MrCanoe,"Hello I'm a 6'1, 260lbs male with Type 2 diabetes. I've been looking into Keto for a while now a...","Hello I'm 6'1, 260lbs male Type 2 diabetes. I've looking Keto way lose weight hopefully lower me...",hello im lbs male type diabetes ive looking keto way lose weight hopefully lower medication im...,"['hello', 'im', 'lbs', 'male', 'type', 'diabetes', 'ive', 'looking', 'keto', 'way', 'lose', 'wei...","['lbs', 'male', 'type', 'diabetes', 'looking', 'keto', 'way', 'lose', 'weight', 'hopefully', 'lo...","['lbs', 'male', 'type', 'diabetes', 'looking', 'keto', 'way', 'lose', 'weight', 'hopefully', 'lo...","['lb', 'male', 'type', 'diabet', 'look', 'keto', 'way', 'lose', 'weight', 'hope', 'lower', 'medi...","['lb', 'male', 'type', 'diabetes', 'looking', 'keto', 'way', 'lose', 'weight', 'hopefully', 'low...",Topic 8,2018-01
2,7nnmor,2018-01-02 08:55:00,Heylola2,I had my a1c checked today and it's at 5.1!!! Three/four months ago it was at 7.9! \nI'm being t...,I a1c checked today it's 5.1!!! Three/four months ago 7.9! I'm taken metformin (and I'll tested ...,i ac checked today its threefour months ago im taken metformin and ill tested months see im ex...,"['i', 'ac', 'checked', 'today', 'its', 'threefour', 'months', 'ago', 'im', 'taken', 'metformin',...","['ac', 'checked', 'today', 'threefour', 'months', 'ago', 'taken', 'metformin', 'ill', 'tested', ...","['ac', 'checked', 'today', 'threefour', 'months', 'ago', 'taken', 'metformin', 'ill', 'tested', ...","['ac', 'check', 'today', 'threefour', 'month', 'ago', 'taken', 'metformin', 'ill', 'test', 'mont...","['ac', 'checked', 'today', 'threefour', 'month', 'ago', 'taken', 'metformin', 'ill', 'tested', '...",Topic 2,2018-01
3,dwcllds,2018-03-26 22:23:00,[deleted],"Hey this is an old post but congrats! This is very motivating.\n\nSide note, how much do you de...","Hey old post congrats! This motivating. Side note, desk cycle for? I got sure reasonable time use .",hey old post congrats this motivating side note desk cycle for i got sure reasonable time use,"['hey', 'old', 'post', 'congrats', 'this', 'motivating', 'side', 'note', 'desk', 'cycle', 'for',...","['old', 'post', 'congrats', 'motivating', 'side', 'note', 'desk', 'cycle', 'got', 'sure', 'reaso...","['old', 'post', 'congrats', 'motivating', 'note', 'desk', 'cycle', 'got', 'sure', 'reasonable', ...","['old', 'post', 'congrat', 'motiv', 'note', 'desk', 'cycl', 'got', 'sure', 'reason', 'time', 'us...","['old', 'post', 'congrats', 'motivating', 'note', 'desk', 'cycle', 'got', 'sure', 'reasonable', ...",Topic 8,2018-03
4,dzokqh6,2018-05-27 16:56:00,Heylola2,I stick my fitbit on my ankle to count 'steps' so I can end up using it the entire time I'm at m...,"I stick fitbit ankle count 'steps' I end entire time I'm desk days, 8+ hours (not usually fast) ...",i stick fitbit ankle count steps i end entire time im desk days hours not usually fast i k step...,"['i', 'stick', 'fitbit', 'ankle', '

In [5]:
list(diabetes2018.columns)

['Post ID',
 'Publish Date',
 'Author',
 'Text',
 'text_gens_nonstop',
 'text_processed',
 'text_tokenized',
 'text_nlk_nonstop',
 'text_nonstop_spacy',
 'text_stemmed',
 'text_lemmatized',
 'Dominant',
 'month_year']

In [6]:
#drop unnecessary columns
diabetes2018 = diabetes2018.drop(columns = ['text_gens_nonstop', 'text_processed', 'text_tokenized', 'text_nlk_nonstop', 'text_nonstop_spacy', 'text_stemmed', 'text_lemmatized'])
diabetes2018

,Post ID,Publish Date,Author,Text,Dominant,month_year
0,7ng7rl,2018-01-01 09:16:00,penguinsarefun,Hey all! Happy New year's! Just wanted to say hi. I was diagnosed with type 2 DM on Thursday. ...,Topic 2,2018-01
1,7nlbda,2018-01-02 00:58:00,MrCanoe,"Hello I'm a 6'1, 260lbs male with Type 2 diabetes. I've been looking into Keto for a while now a...",Topic 8,2018-01
2,7nnmor,2018-01-02 08:55:00,Heylola2,I had my a1c checked today and it's at 5.1!!! Three/four months ago it was at 7.9! \nI'm being t...,Topic 2,2018-01
3,dwcllds,2018-03-26 22:23:00,[deleted],"Hey this is an old post but congrats! This is very motivating.\n\nSide note, how much do you de...",Topic 8,2018-03
4,dzokqh6,2018-05-27 16:56:00,Heylola2,I stick my fitbit on my ankle to count 'steps' so I can end up using it the entire time I'm at m...,Topic 7,2018-05
...,...,...,...,...,...,...
11887,ecv4dtv,2018-12-30 02:02:00,AccidentalDragon,"In 6 mos, my A1c went from 10.5-ish to 6.7, my cholesterol down 30 pts, triglycerides down 90 pt...",Topic 9,2018-12
11888,ecve2zu,2018-12-30 05:56:00,gnilmit,"What everyone else said. I don't even do full keto - I keep my carbs under 50, instead of under ...",Topic 4,2018-12
11889,ecwd91h,2018-12-30 13:32:00,jacobi123,A perfect example of not letting the perfect become the enemy of the good.,Topic 4,2018-12
11890,ecwiejq,2018-12-30 14:31:00,FlowMang,Same. Mine was 9.3. Was down to 5.5 after 90 days and been 5.2 and 5.3 ever since(about a year s...,Topic 7,2018-12


In [7]:
types = diabetes2018.dtypes
types

Post ID         object
Publish Date    object
Author          object
Text            object
Dominant        object
month_year      object
dtype: object

In [8]:
#convert object to string
diabetes2018['Text'] = diabetes2018['Text'].astype(str)

In [9]:
#sentiment analysis with textblob
#https://medium.com/swlh/simple-sentiment-analysis-for-nlp-beginners-and-everyone-else-using-vader-and-textblob-728da3dbe33d

#load the descriptions into textblob
desc_blob = [TextBlob(desc) for desc in diabetes2018['Text']]

#add the sentiment metrics to the dataframe
diabetes2018['tb_Pol'] = [b.sentiment.polarity for b in desc_blob]
diabetes2018['tb_Subj'] = [b.sentiment.subjectivity for b in desc_blob]

diabetes2018.head()

,Post ID,Publish Date,Author,Text,Dominant,month_year,tb_Pol,tb_Subj
0,7ng7rl,2018-01-01 09:16:00,penguinsarefun,Hey all! Happy New year's! Just wanted to say hi. I was diagnosed with type 2 DM on Thursday. ...,Topic 2,2018-01,0.112973,0.552652
1,7nlbda,2018-01-02 00:58:00,MrCanoe,"Hello I'm a 6'1, 260lbs male with Type 2 diabetes. I've been looking into Keto for a while now a...",Topic 8,2018-01,0.005572,0.433556
2,7nnmor,2018-01-02 08:55:00,Heylola2,I had my a1c checked today and it's at 5.1!!! Three/four months ago it was at 7.9! \nI'm being t...,Topic 2,2018-01,0.201414,0.588510
3,dwcllds,2018-03-26 22:23:00,[deleted],"Hey this is an old post but congrats! This is very motivating.\n\nSide note, how much do you de...",Topic 8,2018-03,0.095000,0.437778
4,dzokqh6,2018-05-27 16:56:00,Heylola2,I stick my fitbit on my ankle to count 'steps' so I can end up using it the entire time I'm at m...,Topic 7,2018-05,-0.081250,0.702500


In [10]:
#load VADER
analyzer = SentimentIntensityAnalyzer()

#Add VADER metrics to dataframe
diabetes2018['compound'] = [analyzer.polarity_scores(v)['compound'] for v in diabetes2018['Text']]
diabetes2018['neg'] = [analyzer.polarity_scores(v)['neg'] for v in diabetes2018['Text']]
diabetes2018['neu'] = [analyzer.polarity_scores(v)['neu'] for v in diabetes2018['Text']]
diabetes2018['pos'] = [analyzer.polarity_scores(v)['pos'] for v in diabetes2018['Text']]

diabetes2018.tail(50)

,Post ID,Publish Date,Author,Text,Dominant,month_year,tb_Pol,tb_Subj,compound,neg,neu,pos
11842,ec2t56y,2018-12-18 15:37:00,oldmanpcgamer,"No, my dr hadn’t even prescribed a monitor. I bought one 8 months ago on my own to see what my ...",Topic 6,2018-12,0.253333,0.680000,-0.1857,0.040,0.960,0.000
11843,ec2vxvy,2018-12-18 16:18:00,LetsGoHawks,Take you monitor to the gym. Test one or two times during your workout and again when you get ho...,Topic 4,2018-12,0.446259,0.624490,0.9484,0.053,0.743,0.204
11844,ec3i25s,2018-12-18 22:01:00,kodemizer,Test test test test test. Empirical data to the rescue!,Topic 6,2018-12,0.125000,0.100000,0.5562,0.000,0.715,0.285
11845,ec4i92v,2018-12-19 09:41:00,looonyz,I think the most important thing here for any T2D is to learn what exactly T2D is! Studies with...,Topic 10,2018-12,0.143254,0.532011,0.9645,0.029,0.858,0.113
11846,eclj3z5,2018-12-26 09:41:00,Ketomealsandrecipes,You do not need all those expensive supplements! You need to focus on a well balanced healthy ke...,Topic 5,2018-12,0.218750,0.461574,0.7256,0.012,0.910,0.078
11847,ecojyxt,2018-12-27 12:50:00,alan_s,Read /u/LetsGoHawks excellent advice again. This might help you use your meter more systematical...,Topic 6,2018-12,0.458333,0.625000,0.7506,0.000,0.714,0.286
11848,a7kg6m,2018-12-19 00:03:00,meaniekareenie82,I was told this morning I have T2. I did an oral glucose tolerance test recently and my results ...,Topic 7,2018-12,-0.007143,0.240476,0.6863,0.044,0.845,0.111
11849,ec3q1s7,2018-12-19 01:13:00,FlowMang,"My doctor told me that postprandial BG is more important than fasting BG. Also, what are your A1...",Topic 7,2018-12,0.296970,0.637879,0.8700,0.048,0.835,0.118
11850,ec3rrag,2018-12-19 02:05:00,meaniekareenie82,Is postprandial BG the 2 hour point? I haven’t done an A1C test but I’ll ask about it at my next...,Topic 2,2018-12,0.158333,0.229167,0.9186,0.000,0.793,0.207
11851,ec3vgib,2018-12-19 03:53:00,FlowMang,Postprandial is the 2 hour point. Sorry I wasn’t clear with that term.\nAnother suggestion is to...,Topic 7,2018-12,0.135000,0.503333,0.9055,0.054,0.803,0.143


In [11]:
#drop textblob columns
diabetes2018 = diabetes2018.drop(columns = ['tb_Pol', 'tb_Subj'])
diabetes2018

,Post ID,Publish Date,Author,Text,Dominant,month_year,compound,neg,neu,pos
0,7ng7rl,2018-01-01 09:16:00,penguinsarefun,Hey all! Happy New year's! Just wanted to say hi. I was diagnosed with type 2 DM on Thursday. ...,Topic 2,2018-01,0.5319,0.087,0.797,0.116
1,7nlbda,2018-01-02 00:58:00,MrCanoe,"Hello I'm a 6'1, 260lbs male with Type 2 diabetes. I've been looking into Keto for a while now a...",Topic 8,2018-01,-0.9600,0.108,0.846,0.046
2,7nnmor,2018-01-02 08:55:00,Heylola2,I had my a1c checked today and it's at 5.1!!! Three/four months ago it was at 7.9! \nI'm being t...,Topic 2,2018-01,0.9133,0.071,0.787,0.142
3,dwcllds,2018-03-26 22:23:00,[deleted],"Hey this is an old post but congrats! This is very motivating.\n\nSide note, how much do you de...",Topic 8,2018-03,0.8477,0.048,0.764,0.189
4,dzokqh6,2018-05-27 16:56:00,Heylola2,I stick my fitbit on my ankle to count 'steps' so I can end up using it the entire time I'm at m...,Topic 7,2018-05,0.0000,0.000,1.000,0.000
...,...,...,...,...,...,...,...,...,...,...
11887,ecv4dtv,2018-12-30 02:02:00,AccidentalDragon,"In 6 mos, my A1c went from 10.5-ish to 6.7, my cholesterol down 30 pts, triglycerides down 90 pt...",Topic 9,2018-12,-0.2003,0.112,0.799,0.089
11888,ecve2zu,2018-12-30 05:56:00,gnilmit,"What everyone else said. I don't even do full keto - I keep my carbs under 50, instead of under ...",Topic 4,2018-12,0.6124,0.000,0.958,0.042
11889,ecwd91h,2018-12-30 13:32:00,jacobi123,A perfect example of not letting the perfect become the enemy of the good.,Topic 4,2018-12,0.0263,0.281,0.433,0.286
11890,ecwiejq,2018-12-30 14:31:00,FlowMang,Same. Mine was 9.3. Was down to 5.5 after 90 days and been 5.2 and 5.3 ever since(about a year s...,Topic 7,2018-12,0.8625,0.000,0.884,0.116


In [12]:
#download data with topics and sentiment score
diabetes2018.to_csv('diabetes2018_sentiment.csv', index = False)

In [14]:
#group average sentiment by topic
average_sentiment = diabetes2018.groupby('Dominant')['compound', 'neg', 'neu', 'pos'].mean()
average_sentiment

<ipython-input-14-0b2cbcd6648e>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  average_sentiment = diabetes2018.groupby('Dominant')['compound', 'neg', 'neu', 'pos'].mean()


,compound,neg,neu,pos
Dominant,,,,
Topic 1,0.271651,0.048601,0.815029,0.136376
Topic 10,0.132269,0.063290,0.844545,0.092172
Topic 2,0.174391,0.063918,0.824672,0.111415
Topic 3,0.429658,0.050841,0.683889,0.265261
Topic 4,0.187345,0.067718,0.826011,0.106276
Topic 5,0.341393,0.054070,0.797775,0.148148
Topic 6,0.255392,0.043030,0.854944,0.102017
Topic 7,0.159316,0.053761,0.865654,0.080582
Topic 8,0.149804,0.062311,0.843147,0.094541


In [18]:
average_sentiment.to_csv('diabetes2018_sentiment_by_topic.csv', index = False)

In [19]:
#group by month 
average_sentiment_month = diabetes2018.groupby('month_year')['compound', 'neg', 'neu', 'pos'].mean()
average_sentiment_month

<ipython-input-19-565a24220143>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  average_sentiment_month = diabetes2018.groupby('month_year')['compound', 'neg', 'neu', 'pos'].mean()


,compound,neg,neu,pos
month_year,,,,
2018-01,0.226928,0.062150,0.800472,0.137404
2018-02,0.244699,0.056363,0.800096,0.143530
2018-03,0.231580,0.060406,0.820362,0.119234
2018-04,0.265871,0.058825,0.808032,0.133138
2018-05,0.218155,0.055454,0.821311,0.123239
2018-06,0.226188,0.055158,0.814821,0.130032
2018-07,0.237392,0.056254,0.812128,0.131613
2018-08,0.236485,0.058683,0.802231,0.139078
2018-09,0.239090,0.060026,0.800606,0.139351


In [20]:
average_sentiment_month.to_csv('diabetes2018_sentiment_by_month.csv', index = False)

In [21]:
#sort by authors & dates
diabetes_authors = diabetes2018.sort_values(by=['Author', 'Publish Date'])
diabetes_authors

,Post ID,Publish Date,Author,Text,Dominant,month_year,compound,neg,neu,pos
462,duo9a0o,2018-02-22 13:40:00,#NAME?,You need to do blood tests to determine which type of diabetes you have. They will look for the ...,Topic 10,2018-02,-0.8516,0.160,0.786,0.054
3158,e1wik7j,2018-07-06 10:37:00,#NAME?,Congrats! Now time to lower the range.,Topic 7,2018-07,0.3595,0.202,0.459,0.339
1386,dw853pd,2018-03-24 13:09:00,---DB---,"What insulin do you use? In the US, before insurance, the most common insulin is actually more t...",Topic 10,2018-03,0.0000,0.000,1.000,0.000
1393,dw89ulf,2018-03-24 14:42:00,---DB---,"My deductible is also high, but I was referencing the cost of a single vial. I'm hoping that yo...",Topic 10,2018-03,0.5719,0.000,0.866,0.134
82,7uhcvf,2018-02-01 00:25:00,--_-_o_-_--,I did it. 😀 I reversed my diabetes mellitus type 2. I don't mean to boast but it feels great....,Topic 8,2018-02,0.9964,0.086,0.733,0.180
...,...,...,...,...,...,...,...,...,...,...
11344,e7mjutr,2018-10-12 01:44:00,zoinkydoinky,"When I went on metaformin, I lost my appetite and am still dropping weight. Don't really get hun...",Topic 2,2018-10,0.1189,0.068,0.851,0.081
8002,e7xvwk9,2018-10-17 08:48:00,zoinkydoinky,Sorry to hear this. I was diagnosed two months ago and my stress levels went through the roof. I...,Topic 2,2018-10,0.8942,0.128,0.631,0.241
8004,e7xx0ab,2018-10-17 09:03:00,zoinkydoinky,"I haven't really delved into it that much. But yeah, l completely understand how you feel. When ...",Topic 9,2018-10,0.9734,0.125,0.479,0.397
10260,dxp7h9x,2018-04-20 13:19:00,zylgan,"when doing Keto you can eat out, is it harder yes. But you can order traditional wings most plac...",Topic 1,2018-04,0.2144,0.000,0.960,0.040


In [22]:
diabetes_authors.to_csv('diabetes2018_authors_sentiment.csv', index = False)